# ML Term Paper - Code: Models
by Ann-Christin and Sarah 15/02/2021

In [2]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso
#pd.set_option("display.max_rows", None, "display.max_columns", None)

In [8]:
import os
path = "C:/Users/gust/Documents/ML_term_paper"  # specify your path here
os.chdir(path)
data = pd.read_csv("Data/piaac_red.csv", sep=',', error_bad_lines=False, index_col=False)
data.shape

(230691, 130)

In [10]:
data.head()


,Unnamed: 0,cntry,cntry_e,inpiaac,seqid,cnt_brth,ageg10lfs,age_r,gender_r,j_q03b,...,nfe12jr,nfe12njr,fnfaet12,fnfaet12jr,fnfaet12njr,nfehrsnjr,nfehrsjr,nfehrs,nopaidworkever,unempflag
0,0,USA,USA,1.0,9355.0,NaN,NaN,NaN,Male,NaN,...,Did not participate in NFE for JR reasons,Did not participate in NFE for NJR reasons,Did not participate in formal or non-formal AET,Did not participate in formal or non-formal AE...,Did not participate in formal or non-formal AE...,NaN,NaN,NaN,Has had paid work,0.0
1,1,USA,USA,1.0,10882.0,NaN,NaN,NaN,Male,NaN,...,Participated in NFE for JR reasons,Did not participate in NFE for NJR reasons,Participated in formal and/or non-formal AET,Participated in formal or non-formal AET for J...,Did not participate in formal or non-formal AE...,0.0,30.0,30.0,Has had paid work,0.0
2,2,USA,USA,1.0,12329.0,NaN,NaN,NaN,Female,NaN,...,Did not participate in NFE for JR reasons,Did not participate in NFE for NJR reasons,Did not participate in formal or non-formal AET,Did not participate in formal or non-formal AE...,Did not participate in formal or non-formal AE...,NaN,NaN,NaN,Has had paid work,0.0
3,3,USA,USA,1.0,10711.0,NaN,NaN,NaN,Female,NaN,...,Participated in NFE for JR reasons,Did not participate in NFE for NJR reasons,Participated in formal and/or non-formal AET,Participated in formal or non-formal AET for J...,Did not participate in formal or non-formal AE...,NaN,NaN,8.0,Has had paid work,0.0
4,4,USA,USA,1.0,10383.0,NaN,NaN,NaN,Male,NaN,...,Participated in NFE for JR reasons,Did not participate in NFE for NJR reasons,Participated in formal and/or non-formal AET,Participated in formal or non-formal AET for J...,Did not participate in formal or non-formal AE...,0.0,80.0,80.0,Has had paid work,0.0


In [11]:
# data cleaning
dcl = data.drop(columns = ['Unnamed: 0'])

In [64]:
dcl["skilled"] = np.where(dcl["iscoskil4"]==1, 1, 0)
dcl["semi_white"] = np.where(dcl["iscoskil4"]==2, 1, 0)
dcl["semi_blue"] = np.where(dcl[iscoskil4]==3, 1, 0)
dcl["low_skilled"] = np.where(dcl[iscoskil4]==4, 1, 0)


In [67]:
dcl = dcl[dcl.iscoskil4 != 'N']
dcl = dcl[dcl.iscoskil4 != 'U']
dcl = dcl[dcl.iscoskil4 != 'A']

In [72]:
dcl.dtypes

cntry           object
cntry_e         object
inpiaac        float64
seqid          float64
cnt_brth        object
                ...   
unempflag      float64
skilled          int32
semi_white       int32
semi_blue        int32
low_skilled      int32
Length: 133, dtype: object

In [78]:
cate = dcl.select_dtypes(include=['object'])
cate = cate.drop(columns = ['iscoskil4'])
cate = list(set(cate))
cate

['isced_hf_c',
 'b_q12a',
 'd_q09',
 'nfe12jr',
 'b_q16',
 'b_q12e',
 'pared',
 'd_q13c',
 'b_q15b',
 'gender_r',
 'b_d12h',
 'c_q02a',
 'fnfaet12njr',
 'b_q11',
 'b_q04a',
 'b_q14a',
 'isco1c',
 'b_q01b',
 'g_q05d',
 'nopaidworkever',
 'fnfaet12jr',
 'fe12',
 'computerexperience',
 'c_d09',
 'd_q16d5',
 'b_q02a_t2',
 'b_q26b',
 'd_q12a',
 'b_q01c1_c',
 'd_q05b1_c',
 'g_q05a',
 'g_q05f',
 'b_d01d_c',
 'nfe12njr',
 'g_q08',
 'd_q12b',
 'c_d06',
 'ageg10lfs',
 'b_q10b',
 'd_q04_t',
 'b_q02a_t1',
 'b_q02a',
 'b_q14b',
 'c_q08c1_c',
 'b_q01a',
 'f_q07a',
 'b_q26a_t',
 'isic1c',
 'c_q02c',
 'b_q15a',
 'b_q10c',
 'b_q12c',
 'isic1l',
 'd_q06c',
 'b_q13',
 'cnt_h',
 'b_q15c',
 'edcat8',
 'g_q05c',
 'g_q07',
 'b_q20b',
 'c_d05',
 'b_q01a_t',
 'd_q06b',
 'd_q06a',
 'b_q10a',
 'd_q07a',
 'faet12njr',
 'cnt_brth',
 'j_q04c1_c',
 'd_q14',
 'b_q12g',
 'g_q06',
 'g_q05g',
 'd_q07b_c',
 'd_q04',
 'cntry',
 'faet12jr',
 'g_q05h',
 'd_q12c',
 'd_q05a1_c',
 'f_q07b',
 'isco1l',
 'g_q04',
 'aetpop',
 'b_

In [82]:
num = dcl.select_dtypes(include=['float64', "int32"])
num = num.loc[:, ~num.columns.str.startswith('b_q12')]
num_list = list(set(num))
num_list

['nfehrsjr',
 'semi_blue',
 'yrsqual_t',
 'c_q10a',
 'earnmthallppp',
 'b_q04b',
 'skilled',
 'd_q05a2',
 'b_q19a',
 'nfehrsnjr',
 'yrsget',
 'j_q03b_c',
 'nfehrs',
 'unempflag',
 'ictwork',
 'earnmthselfppp',
 'learnatwork',
 'inpiaac',
 'age_r',
 'earnmthbonusppp',
 'readytolearn',
 'd_q16b',
 'c_q08c2',
 'seqid',
 'j_q03b',
 'd_q05b2',
 'c_q09',
 'low_skilled',
 'yrsqual',
 'racethn_5cat',
 'icthome',
 'semi_white']

In [79]:
# generate dummies
dummies = [pd.get_dummies(dcl[i], prefix=i, drop_first = True) for i in [cate]]
dummies = pd.DataFrame(dummies)

In [80]:
# define X and y
on_job = dcl["b_q12d"] #define y here
open_educ = dcl["b_q12f"]
X = num.join(dummies)
X = pd.DataFrame(np.ascontiguousarray(X.values), columns = X.columns)

In [81]:
# run OLS
import statsmodels.api as sm
est = sm.OLS(on_job, num, missing='drop').fit()
X = sm.add_constant(num)
est.summary()

ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:
# feature importance

imp = pd.DataFrame(rf.feature_importances_, columns = ['Feature importance'])
imp['Feature'] = list(X.columns)
imp.sort_values(by=['Feature importance'], ascending=False).reset_index(drop = True)


In [ ]:
# lasso

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1) 
pipe = Pipeline([('scaler', StandardScaler()),
                 ('lasso', 
                  Lasso(alpha=0.005, fit_intercept=True, normalize=False, copy_X=False, max_iter=10000))]) 
# note alpha == lambda in the lecture notes; we just take some value - it is not the optimal! 
# again if you simply follow the defaults, it is not necessary to specify the options
pipe.fit(X_train, y_train)
print('In-sample Rsq: % .4f'
     % pipe.score(X_train, y_train))
print('Out-of-sample Rsq: % .4f'
     % pipe.score(X_test, y_test))
#print(regr.coef_)

In [ ]:
# compare to 